In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import transformers
import load_data
import torch
from torch import nn
from transformers import BertModel, BertTokenizer, CamembertModel, CamembertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
import numpy as np

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu') # GPU ou CPU
print(torch.cuda.is_available())
# Chargement des données
transcr_path='paco-cheese/transcr'
data=load_data.load_all_ipus(folder_path=transcr_path,load_words=True)



C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True


,ipu_id,speaker,start_ipu,stop_ipu,text_ipu,is_main_speaker,turn_at_start,turn_after,turn_start_word,yield_at_end,request_at_start,start_words,stop_words,duration,text_words,request_after_word,turn_after_word,is_ipu_end,dyad
275,33,AA,77.42,79.400,étendu devant la voiture il y avait un âne,True,False,True,79.4,True,False,78.78,78.89,0.11,y,False,False,False,transcr\AAOR
276,33,AA,77.42,79.400,étendu devant la voiture il y avait un âne,True,False,True,79.4,True,False,78.89,79.00,0.11,avait,False,False,False,transcr\AAOR
277,33,AA,77.42,79.400,étendu devant la voiture il y avait un âne,True,False,True,79.4,True,False,79.00,79.17,0.17,un,False,False,False,transcr\AAOR
278,33,AA,77.42,79.400,étendu devant la voiture il y avait un âne,True,False,True,79.4,True,False,79.17,79.40,0.23,âne,False,False,True,transcr\AAOR
279,34,OR,81.30,82.270,d'accord super,True,True,False,NaN,False,False,81.30,81.82,0.52,d'accord,False,False,False,transcr\AAOR
280,34,OR,81.30,82.270,d'accord super,True,True,False,NaN,False,False,82.00,82.27,0.27,super,False,False,True,transcr\AAOR
281,35,AA,83.64,84.120,voilà,False,False,False,NaN,False,False,83.64,84.12,0.48,voilà,False,False,True,transcr\AAOR
282,36,OR,86.76,89.382,mh attends j'avais plein de trucs à dire en mê...,True,False,False,NaN,False,True,86.76,87.59,0.83,mh,True,False,False,transcr\AAOR
283,36,OR,86.76,89.382,mh attends j'avais plein de trucs à dire en mê...,True,False,False,NaN,False,True,87.59,87.74,0.15,attends,False,False,False,transcr\AAOR
284,36,OR,86.76,89.382,mh attends j'avais plein de trucs à dire en mê...,True,False,False,NaN,False,True,87.74,87.83,0.09,j',False,False,False,transcr\AAOR


In [1]:
data[275:285]

NameError: name 'data' is not defined

In [2]:
def create_sequences(text_series, window_size=5):
    sequences = []
    for i in range(window_size, len(text_series)):
        # Prendre 20 mots précédents et le mot courant pour former la séquence
        sequence = text_series[i - window_size:i+1]
        # Convert all items to strings
        sequence = [str(word) for word in sequence]
        sequences.append(' '.join(sequence))
    return sequences

# creation des y à prédire

In [3]:
# Load the data into a DataFrame
df = pd.DataFrame(data)

# Create a new list 'y' to store 1s and 0s based on speaker change
y = [0]  # Initialize the first element to 0, as there's no previous speaker to compare with.

# Iterate over the DataFrame, starting from the second row
for i in range(1, len(df)-1):
    # Check if the current speaker is different from the previous one
    if df['speaker'][i] != df['speaker'][i+1]:
        y.append(1)  # Speaker changed
    else:
        y.append(0)  # Speaker did not change

y
indices = [i for i, x in enumerate(y) if x == 1]
print(len(indices))
print(indices)

11497
[1, 204, 278, 280, 281, 295, 296, 304, 307, 314, 322, 333, 334, 335, 342, 345, 360, 370, 374, 382, 398, 426, 427, 431, 446, 454, 458, 479, 480, 486, 497, 503, 504, 569, 570, 603, 606, 620, 623, 642, 645, 651, 655, 668, 670, 673, 690, 691, 708, 711, 733, 758, 759, 784, 785, 820, 824, 859, 861, 868, 878, 879, 925, 928, 953, 955, 957, 991, 994, 1009, 1021, 1024, 1033, 1035, 1042, 1047, 1066, 1067, 1073, 1077, 1078, 1089, 1090, 1106, 1109, 1116, 1117, 1120, 1126, 1138, 1140, 1158, 1162, 1185, 1189, 1192, 1197, 1210, 1217, 1223, 1224, 1234, 1244, 1248, 1268, 1277, 1279, 1302, 1304, 1343, 1344, 1365, 1369, 1376, 1378, 1382, 1388, 1394, 1411, 1419, 1431, 1432, 1433, 1448, 1480, 1492, 1494, 1501, 1502, 1512, 1514, 1523, 1526, 1551, 1561, 1585, 1586, 1599, 1604, 1608, 1609, 1610, 1662, 1683, 1706, 1743, 1747, 1759, 1766, 1774, 1796, 1809, 1834, 1836, 1843, 1858, 1861, 1874, 1875, 1882, 1883, 1889, 1891, 1897, 1906, 1934, 1958, 1959, 1961, 1987, 1999, 2010, 2021, 2025, 2030, 2031, 2036, 20

In [4]:
label_encoder = LabelEncoder()
data['turn_after_word_encoded'] = label_encoder.fit_transform(data['is_ipu_end'])

# Sélection des caractéristiques et des étiquettes
features = data['text_words'].to_numpy().reshape(-1).tolist()  # Assurez-vous que c'est une liste de mots
import numpy as np

# Replace NaN values with a placeholder string
features = [word if not pd.isnull(word) else '[UNK]' for word in features]

# Now, you can create sequences


labels = data['turn_after_word_encoded'].tolist()  # Convertir les étiquettes en liste aussi

# Calcul de l'index pour la séparation en respectant l'ordre
train_size = int(len(features) * 0.8)
train_size=train_size//25
# Séparation en ensembles d'entraînement et de test
X_train = features[:train_size]
y_train = y[:train_size]


indices = [i for i, x in enumerate(y_train) if x == 1]
print(indices)


# on prends 4 fois plus de données de test afin de s'assurer de l'efficacité du modèle
X_test = features[len(features)-train_size*4:]
y_test = labels[len(features)-train_size*4:]

window_size = 5
# Appliquer la fonction pour créer des séquences pour l'entraînement et le test
X_train = create_sequences(X_train)
X_test = create_sequences(X_test)

# Les étiquettes doivent être ajustées pour correspondre aux séquences
y_train = y_train[window_size:]  # Les étiquettes commencent là où nous avons une séquence complète
y_test = y_test[window_size:]    # Idem pour l'ensemble de test

# Vérification des dimensions
print(f"Nombre de séquences d'entraînement: {len(X_train)}, étiquettes: {len(y_train)}")
print(f"Nombre de séquences de test: {len(X_test)}, étiquettes: {len(y_test)}")


[1, 204, 278, 280, 281, 295, 296, 304, 307, 314, 322, 333, 334, 335, 342, 345, 360, 370, 374, 382, 398, 426, 427, 431, 446, 454, 458, 479, 480, 486, 497, 503, 504, 569, 570, 603, 606, 620, 623, 642, 645, 651, 655, 668, 670, 673, 690, 691, 708, 711, 733, 758, 759, 784, 785, 820, 824, 859, 861, 868, 878, 879, 925, 928, 953, 955, 957, 991, 994, 1009, 1021, 1024, 1033, 1035, 1042, 1047, 1066, 1067, 1073, 1077, 1078, 1089, 1090, 1106, 1109, 1116, 1117, 1120, 1126, 1138, 1140, 1158, 1162, 1185, 1189, 1192, 1197, 1210, 1217, 1223, 1224, 1234, 1244, 1248, 1268, 1277, 1279, 1302, 1304, 1343, 1344, 1365, 1369, 1376, 1378, 1382, 1388, 1394, 1411, 1419, 1431, 1432, 1433, 1448, 1480, 1492, 1494, 1501, 1502, 1512, 1514, 1523, 1526, 1551, 1561, 1585, 1586, 1599, 1604, 1608, 1609, 1610, 1662, 1683, 1706, 1743, 1747, 1759, 1766, 1774, 1796, 1809, 1834, 1836, 1843, 1858, 1861, 1874, 1875, 1882, 1883, 1889, 1891, 1897, 1906, 1934, 1958, 1959, 1961, 1987, 1999, 2010, 2021, 2025, 2030, 2031, 2036, 2041, 20

In [5]:
print(X_train)
print(y_train[365:375])

['tu as mh ouais si tu', 'as mh ouais si tu veux', 'mh ouais si tu veux frog', 'ouais si tu veux frog joke', 'si tu veux frog joke un', 'tu veux frog joke un jour', 'veux frog joke un jour un', 'frog joke un jour un ingénieur', 'joke un jour un ingénieur traversait', 'un jour un ingénieur traversait la', 'jour un ingénieur traversait la rue', 'un ingénieur traversait la rue quand', 'ingénieur traversait la rue quand une', 'traversait la rue quand une grenouille', "la rue quand une grenouille l'", "rue quand une grenouille l' interpela", "quand une grenouille l' interpela et", "une grenouille l' interpela et dit", "grenouille l' interpela et dit si", "l' interpela et dit si tu", "interpela et dit si tu m'", "et dit si tu m' embrasses", "dit si tu m' embrasses je", "si tu m' embrasses je me", "tu m' embrasses je me transforme", "m' embrasses je me transforme en", 'embrasses je me transforme en belle', 'je me transforme en belle princesse', 'me transforme en belle princesse il', "transfor

In [7]:
# Paramètres
model_name = 'camembert-base'
max_length = 256
batch_size = 16
epochs = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Tokenizer pour CamemBERT
tokenizer = CamembertTokenizer.from_pretrained(model_name)

# Classe de Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }
#y_train = y_train[window_size:] if len(y_train) > window_size else y_train
#y_test = y_test[window_size:] if len(y_test) > window_size else y_test

# Création des datasets et dataloaders
train_dataset = TextDataset(X_train, y_train, tokenizer, max_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TextDataset(X_test, y_test, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
print(len(X_train))
print(len(y_train))

3532
3532


# Entrainement

In [12]:
from transformers import CamembertTokenizer, CamembertForSequenceClassification
# Charger le modèle CamemBERT pour la classification de séquences
model = CamembertForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.to(device)

print(len(train_loader))
# Optimiseur et planning de taux d'apprentissage
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)
# Poids pour les classes
# Poids pour la classe 0 est 1 (classe majoritaire)
# Poids pour la classe 1 est 10 (classe minoritaire)
class_weights = torch.tensor([1.0, 15.0], device=device)

# Créer l'objet de perte avec des poids
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

# Fonction d'entraînement
# Fonction d'entraînement avec perte pondérée
def train_epoch(model, data_loader, optimizer, device, scheduler, loss_fn):
    model.train()
    losses = []
    correct_predictions = 0

    for d in data_loader:
        #afficher shape d
        #print(d["input_ids"].shape)
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)

        # Efface les gradients existants avant de calculer les gradients pour les nouvelles prédictions
        model.zero_grad()

        # Effectuer une prédiction (forward pass)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # Calculer la perte en utilisant les poids
        loss = loss_fn(outputs.logits, labels)

        # Calculer les prédictions
        _, preds = torch.max(outputs.logits, dim=1)
        correct_predictions += torch.sum(preds == labels)

        # Enregistrer la perte
        losses.append(loss.item())

        # Effectuer une rétropropagation (backward pass)
        loss.backward()

        # Clipper les gradients pour éviter l'explosion des gradients
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Mettre à jour les poids du modèle
        optimizer.step()

        # Mettre à jour le planning du taux d'apprentissage
        scheduler.step()

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)


# Boucle d'entraînement
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_loader,
        optimizer,
        device,
        scheduler,
        loss_fn
    )

    print(f'Train loss {train_loss} accuracy {train_acc}')



Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


221
Epoch 1/2
----------
Train loss 0.6780651298043955 accuracy 0.7953001132502832
Epoch 2/2
----------
Train loss 0.5733908433719997 accuracy 0.8202151755379389


# score f1 et confusion matrix sur le test

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix


def calculate_f1_and_confusion_matrix(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, preds = torch.max(logits, dim=1)

            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    f1 = f1_score(all_labels, all_preds, average='weighted')
    conf_matrix = confusion_matrix(all_labels, all_preds)

    return f1, conf_matrix

# Utilisation de la fonction
f1_score_test, confusion_matrix_test = calculate_f1_and_confusion_matrix(model, test_loader, device)
print(f'Test F1 Score: {f1_score_test}')
print(f'Confusion Matrix:\n{confusion_matrix_test}')

# Calculer et afficher les informations spécifiques
total_class_0 = np.sum(confusion_matrix_test[0])
total_class_1 = np.sum(confusion_matrix_test[1])
detected_class_0 = confusion_matrix_test[0, 0]  # Vrais positifs pour la classe 0
detected_class_1 = confusion_matrix_test[1, 1]  # Vrais positifs pour la classe 1

print(f'Nombre d\'éléments de classe 0 détectés : {detected_class_0} sur {total_class_0}')
print(f'Nombre d\'éléments de classe 1 détectés : {detected_class_1} sur {total_class_1}')


Test F1 Score: 0.8378712041163751
Confusion Matrix:
[[11335  1016]
 [ 1219   573]]
Nombre d'éléments de classe 0 détectés : 11335 sur 12351
Nombre d'éléments de classe 1 détectés : 573 sur 1792


# prédictions sur quelques données afin de voir ce qu'il se passe : 

In [ ]:
import torch
from transformers import CamembertTokenizer

# Initialize the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Assume 'data' is your list of French sentences


# Process data and predict probabilities
all_probs = []
data=X_train[273:277]
with torch.no_grad():
    for sentence in data:
        # Tokenize the sentence
        print(sentence)
        inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=512)
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        # Predict
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Convert logits to probabilities
        probs = torch.softmax(logits, dim=1)
        all_probs.append(probs.cpu().numpy())

# 'all_probs' now contains the probabilities for each sentence in your list

        print(outputs)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


voiture il y avait un âne
SequenceClassifierOutput(loss=None, logits=tensor([[ 0.4949, -0.3274]], device='cuda:0'), hidden_states=None, attentions=None)
il y avait un âne d'accord
SequenceClassifierOutput(loss=None, logits=tensor([[ 0.3227, -0.3151]], device='cuda:0'), hidden_states=None, attentions=None)
y avait un âne d'accord super
SequenceClassifierOutput(loss=None, logits=tensor([[-0.5121,  0.3384]], device='cuda:0'), hidden_states=None, attentions=None)
avait un âne d'accord super voilà
SequenceClassifierOutput(loss=None, logits=tensor([[-0.2058, -0.1855]], device='cuda:0'), hidden_states=None, attentions=None)


In [ ]:
indices = [i for i, x in enumerate(y_train) if x == 1]
indices

[199,
 273,
 275,
 276,
 290,
 291,
 299,
 302,
 309,
 317,
 328,
 329,
 330,
 337,
 340,
 355,
 365,
 369,
 377,
 393,
 421,
 422,
 426,
 441,
 449,
 453,
 474,
 475,
 481,
 492,
 498,
 499,
 564,
 565,
 598,
 601,
 615,
 618,
 637,
 640,
 646,
 650,
 663,
 665,
 668,
 685,
 686,
 703,
 706,
 728,
 753,
 754,
 779,
 780,
 815,
 819,
 854,
 856,
 863,
 873,
 874,
 920,
 923,
 948,
 950,
 952,
 986,
 989,
 1004,
 1016,
 1019,
 1028,
 1030,
 1037,
 1042,
 1061,
 1062,
 1068,
 1072,
 1073,
 1084,
 1085,
 1101,
 1104,
 1111,
 1112,
 1115,
 1121,
 1133,
 1135,
 1153,
 1157,
 1180,
 1184,
 1187,
 1192,
 1205,
 1212,
 1218,
 1219,
 1229,
 1239,
 1243,
 1263,
 1272,
 1274,
 1297,
 1299,
 1338,
 1339,
 1360,
 1364,
 1371,
 1373,
 1377,
 1383,
 1389,
 1406,
 1414,
 1426,
 1427,
 1428,
 1443,
 1475,
 1487,
 1489,
 1496,
 1497,
 1507,
 1509,
 1518,
 1521,
 1546,
 1556,
 1580,
 1581,
 1594,
 1599,
 1603,
 1604,
 1605,
 1657,
 1678,
 1701,
 1738,
 1742,
 1754,
 1761,
 1769,
 1791,
 1804,
 1829,
 183

In [ ]:

print(y_train[indices[1]-5:indices[1]+5])
X_train[indices[1]-5:indices[1]+5]

[0, 0, 0, 0, 0, 1, 0, 1, 1, 0]


['pour lui étendu devant la voiture',
 'lui étendu devant la voiture il',
 'étendu devant la voiture il y',
 'devant la voiture il y avait',
 'la voiture il y avait un',
 'voiture il y avait un âne',
 "il y avait un âne d'accord",
 "y avait un âne d'accord super",
 "avait un âne d'accord super voilà",
 "un âne d'accord super voilà mh"]